# Actividad - Proyecto práctico (Versión optimizada para máximo rendimiento)

> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1: Granizo, Mateo
*   Alumno 2: Maiolo, Pablo
*   Alumno 3: Miglino, Diego


---
## **PARTE 1** - Instalación y requisitos previos

> NOTA: Esta versión usa parámetros optimizados para máximo rendimiento en Space Invaders.

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)

> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. Instalar Anaconda
2. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.

```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
pip install jupyter
```

3. Abrir la notebook con *jupyter-notebook*.

```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [ ]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/TP_Grupal"
mount='./'

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False
print(IN_COLAB)

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

---
### 1.4. Instalar librerías necesarias

In [ ]:
# ejecutar solo la primera vez..
# Instalación mejorada de atari-py usando el fork de OpenAI

if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/openai/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.12
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/openai/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.5.3

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [ ]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory, PrioritizedMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

#### Configuración base

In [ ]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4  # Captura 4 frames consecutivos para percibir movimiento

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

print("Numero de acciones disponibles: " + str(nb_actions))
print("Formato de las observaciones: " + str(env.observation_space))

In [ ]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # Convertir a escala de grises
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # Guardar como uint8 para ahorrar memoria

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.  # Normalizar a [0, 1]
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)  # Recortar recompensas para estabilizar el aprendizaje

1. Implementación de la red neuronal

In [ ]:
# Modelo mejorado con arquitectura más profunda
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()

if K.image_data_format() == 'channels_last':
    # (width, height, channels)
    model.add(Permute((2, 3, 1), input_shape=input_shape))
elif K.image_data_format() == 'channels_first':
    # (channels, width, height)
    model.add(Permute((1, 2, 3), input_shape=input_shape))
else:
    raise RuntimeError('Unknown image_dim_ordering.')

# Primera capa convolucional
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(BatchNormalization())  # Normalización por lotes para estabilizar el entrenamiento
model.add(Activation('relu'))

# Segunda capa convolucional
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Tercera capa convolucional
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Cuarta capa convolucional adicional para mayor capacidad
model.add(Convolution2D(128, (3, 3), strides=(1, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Aplanar para capas densas
model.add(Flatten())

# Primera capa densa
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))  # Dropout para evitar sobreajuste

# Segunda capa densa
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Capa de salida
model.add(Dense(nb_actions))
model.add(Activation('linear'))

print(model.summary())

2. Implementación de la solución DQN

In [ ]:
# Memoria para almacenar la experiencia del agente
# Usamos una memoria grande para permitir un aprendizaje estable
memory = SequentialMemory(limit=100000, window_length=WINDOW_LENGTH)

# Procesador para las observaciones
processor = AtariProcessor()

# Política de exploración con decaimiento lineal
# Comenzamos con exploración completa (1.0) y reducimos gradualmente a 0.05
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps',
                              value_max=1.0, value_min=0.05, value_test=0.01,
                              nb_steps=100000)  # Decaimiento más lento para mejor exploración

# Definición del agente DQN
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy,
               memory=memory, processor=processor,
               nb_steps_warmup=10000,  # Más pasos de calentamiento para llenar la memoria
               gamma=0.99,  # Factor de descuento alto para valorar recompensas futuras
               target_model_update=10000,  # Actualización menos frecuente de la red objetivo
               train_interval=4,  # Entrenar cada 4 pasos para estabilidad
               delta_clip=1.0)  # Recortar el error delta para evitar explosiones de gradiente

# Compilación del agente con optimizador RMSprop (mejor para DQN en Atari)
dqn.compile(RMSprop(learning_rate=0.00025, rho=0.95, epsilon=0.01), metrics=['mae'])

In [ ]:
# Entrenamiento del agente
weights_filename = 'dqn_max_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_max_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_max_{}_log.json'.format(env_name)

# Callbacks para guardar checkpoints y logs
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=10000)]
callbacks += [FileLogger(log_filename, interval=1000)]

# Entrenamiento por 200,000 pasos (mucho más largo para mejor rendimiento)
dqn.fit(env, callbacks=callbacks, nb_steps=200000, log_interval=10000, visualize=False)

# Guardar pesos finales
dqn.save_weights(weights_filename, overwrite=True)

In [ ]:
# Test de n episodios para calcular la recompensa final
weights_filename = 'dqn_max_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=True)

3. Justificación de los parámetros seleccionados y de los resultados obtenidos

### Justificación de los parámetros seleccionados

En esta implementación optimizada para máximo rendimiento, se han realizado numerosas mejoras respecto a la versión original:

1. **Arquitectura de red neuronal mejorada**:
   - Se añadió una capa convolucional adicional (128 filtros) para capturar patrones más complejos.
   - Se incorporó BatchNormalization después de cada capa para estabilizar y acelerar el entrenamiento.
   - Se añadió una capa densa adicional (256 neuronas) para mayor capacidad de representación.
   - Se implementó Dropout (20%) para prevenir el sobreajuste.

2. **Memoria de experiencia mucho más grande**:
   - Aumentada de 500 a 100,000 experiencias (200 veces más grande).
   - Permite al agente aprender de una variedad mucho mayor de situaciones y mantener experiencias valiosas por más tiempo.

3. **Exploración más efectiva**:
   - Decaimiento de epsilon más lento (100,000 pasos vs 100 originales).
   - Valor mínimo de epsilon reducido a 0.05 para mantener algo de exploración incluso al final.
   - Valor de test reducido a 0.01 para evaluación más determinista.

4. **Entrenamiento mucho más extenso**:
   - 200,000 pasos vs 1,500 originales (más de 130 veces más largo).
   - Permite al agente desarrollar estrategias mucho más sofisticadas.

5. **Optimizador RMSprop en lugar de Adam**:
   - RMSprop con parámetros específicos (learning_rate=0.00025, rho=0.95) ha demostrado mejor rendimiento en DQN para juegos Atari según la literatura.

6. **Otros parámetros optimizados**:
   - nb_steps_warmup aumentado a 10,000 para llenar adecuadamente la memoria antes de comenzar el aprendizaje.
   - target_model_update aumentado a 10,000 para mayor estabilidad.
   - train_interval=4 para actualizar la red cada 4 pasos, siguiendo las mejores prácticas de DQN.
   - delta_clip=1.0 para evitar actualizaciones demasiado grandes que podrían desestabilizar el entrenamiento.

### Resultados esperados

Con estas mejoras, esperamos que el agente logre una puntuación significativamente superior al requisito mínimo de 20 puntos. Basándonos en implementaciones similares en la literatura, este modelo optimizado debería ser capaz de alcanzar puntuaciones en el rango de 500-1000 puntos en Space Invaders, lo que representa un rendimiento muy competente.

Las mejoras implementadas siguen las mejores prácticas establecidas en los artículos seminales sobre DQN y sus variantes, particularmente las recomendaciones de DeepMind para entornos Atari. La arquitectura más profunda, junto con las técnicas de regularización y el entrenamiento extenso, deberían permitir al agente desarrollar estrategias sofisticadas para maximizar su puntuación en Space Invaders.

---